# ==========================================
# *DevOps Tool Calling Agent (Ollama + LangChain)*
# ==========================================

In [3]:
# ==========================================
# ReAct Tool Agent (phi3 Compatible)
# ==========================================

from langchain_ollama import ChatOllama

# -----------------------------
# 1️⃣ Initialize LLM
# -----------------------------

llm = ChatOllama(
    model="phi3:latest",
    temperature=0.1
)

# -----------------------------
# 2️⃣ Define Tools
# -----------------------------

def analyze_error(error: str) -> str:
    return f"Root cause analysis for: {error}"


def generate_fix(issue: str) -> str:
    return f"Fix plan for: {issue}"


def monitoring_strategy(service: str) -> str:
    return f"Monitoring strategy for: {service}"


tools = {
    "analyze_error": analyze_error,
    "generate_fix": generate_fix,
    "monitoring_strategy": monitoring_strategy
}

# -----------------------------
# 3️⃣ Tool Selection Step
# -----------------------------

def select_tools(user_input: str):

    decision_prompt = f"""
You are a DevOps AI agent.

Available tools:
- analyze_error
- generate_fix
- monitoring_strategy

Select required tools.
Return only tool names separated by commas.
Do not explain.

User request:
{user_input}
"""

    response = llm.invoke(decision_prompt).content.lower()

    selected = []
    for name in tools.keys():
        if name in response:
            selected.append(name)

    return selected


# -----------------------------
# 4️⃣ Agent Execution Loop
# -----------------------------

def run_agent(user_input: str):

    print("\nUser:", user_input)

    selected_tools = select_tools(user_input)

    print("Selected tools:", selected_tools)

    observations = []

    for tool_name in selected_tools:
        result = tools[tool_name](user_input)
        observations.append(f"{tool_name} output: {result}")

    final_prompt = f"""
User request:
{user_input}

Tool results:
{chr(10).join(observations)}

Provide structured DevOps response.
"""

    final_answer = llm.invoke(final_prompt).content

    return final_answer


# -----------------------------
# 5️⃣ Run Example
# -----------------------------

if __name__ == "__main__":

    output = run_agent("""
My Kubernetes pod shows CrashLoopBackOff error.
Provide root cause, fix plan and monitoring strategy.
""")

    print("\n=========== FINAL OUTPUT ===========\n")
    print(output)



User: 
My Kubernetes pod shows CrashLoopBackOff error.
Provide root cause, fix plan and monitoring strategy.

Selected tools: ['analyze_error', 'generate_fix', 'monitoring_strategy']

=========== FINAL OUTPUT ===========

**Root Cause Analysis Report:**  

The `CrashLoopBackOff` status indicates that the Kubernetes pod is repeatedly crashing because it cannot start successfully within a certain amount of time, typically due to issues with container runtime or application code/configuration problems. Common causes include misconfigured environment variables, resource constraints (like memory limits), incompatible software versions between dependencies, and flaky tests leading to unstable builds.

**Fix Plan:**  
1. Check the pod's logs using `kubectl logs <pod-name>` for immediate insights into what might be failing during startup.
2. Validate that all environment variables are correctly set in your Pod specification and match those expected by your application, if any such expectation